In [ ]:
import pandas as pd
import numpy as np
import statistics
from tqdm import tqdm
from matplotlib import pyplot as plt

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
from sklearn.pipeline import Pipeline 

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from catboost import CatBoostClassifier,Pool

In [ ]:
from Boruta_Shap import BorutaShap

In [ ]:
import shap

In [ ]:
import optuna

DATA PREPARATION and PCA

From now on, suppose you have already a train-validation-test split of the data. Image features have been already extracted and joined to clinical features in respective data files. Minmax 0-1 normalization and zero-mean on image features has been already effected. Data files are .csv. Files contain also a binary 'OUTCOME' column.

In [ ]:
filenametrain='train_tenth_fold.csv'
filenamevalid = 'valid_tenth_fold.csv'
filenametest= 'test.csv'

In [ ]:
dftrain = pd.read_csv(filenametrain, header = 0, names=columns_eng)
dfvalid = pd.read_csv(filenamevalid, header = 0, names=columns_eng)
dftest = pd.read_csv(filenametest, header = 0, names=columns_eng)

read  csv files as dataframes with pandas. Check colums in one of the files

In [ ]:
dftrain.columns

check outcome stratification

In [ ]:
dftrain['OUTCOME'].value_counts()

In [ ]:
dfvalid['OUTCOME'].value_counts()

In [ ]:
dftest['OUTCOME'].value_counts()

set index on patient ID and extract outcomes

In [ ]:
dftrain = dftrain.set_index('patient_id')
dfvalid = dfvalid.set_index('patient_id')
dftest = dftest.set_index('patient_id'')

In [ ]:
y_train = dftrain.pop('OUTCOME').values
y_valid = dfvalid.pop('OUTCOME').values
y_test = dftest.pop('OUTCOME').values

separate image features (40, in our case), for PCA.
5 PCA are extracted on the training set.
The PCA transform obtained in this way is applied also to the validation set and to the training set.
The resulting features are then attached again to their respective instances

In [ ]:
dfcp=dftrain[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40']]
dfcpv=dfvalid[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40']]
dfcpt=dftest[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40']]


In [ ]:
pca = PCA(n_components=5,svd_solver='full')

full solver is utilized to assure perfect reproducibility

In [ ]:
pca.fit(dfcp)

In [ ]:
dfpca=pd.DataFrame(pca.transform(dfcp), index=dfcp.index)
dfpcav=pd.DataFrame(pca.transform(dfcpv), index=dfcpv.index)
dfpcat=pd.DataFrame(pca.transform(dfcpt), index=dfcpt.index)

In [ ]:
dfpca.columns = ['Feature_CT_1','Feature_CT_2','Feature_CT_3','Feature_CT_4','Feature_CT_5']
dfpcav.columns = ['Feature_CT_1','Feature_CT_2','Feature_CT_3','Feature_CT_4','Feature_CT_5']
dfpcat.columns = ['Feature_CT_1','Feature_CT_2','Feature_CT_3','Feature_CT_4','Feature_CT_5']

In [ ]:
dftrain=dftrain.drop(columns=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40'])
dfvalid=dfvalid.drop(columns=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40'])
dftest=dftest.drop(columns=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40'])

In [ ]:
dftrain=pd.merge(dftrain, dfpca, on='patient_ID', how='inner')
dfvalid=pd.merge(dfvalid, dfpcav, on='patient_ID', how='inner')
dftest=pd.merge(dftest, dfpcat, on='patient_ID', how='inner')

in order to avoid compatibility problems, replace NaN with np.NaN and convert dtypes.

In [ ]:
X_train.replace('NaN',np.NaN,inplace=True)
X_valid.replace('NaN',np.NaN,inplace=True)
X_test.replace('NaN',np.NaN,inplace=True)

In [ ]:
X_train = X_train.convert_dtypes()
X_valid = X_valid.convert_dtypes()
X_test = X_test.convert_dtypes()

check for missing data

In [ ]:
print(X_train.isnull().sum())

END of DATA PREPARATION and PCA

FEATURE SELECTION

start building a preliminary CatBoost Classifier

run a dummy classifier on the training set to evaluate the learning rate. We fixed iterations at 700 in this phase.

In [ ]:
cbmodel = CatBoostClassifier(iterations=700, verbose=1000)

split 0.8:0.2 the training set

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X_train, y_train, test_size=0.2)

median imputation avoiding knowledge leakage, therefore keeping separate training and validation. 

In [ ]:
median_imputer=SimpleImputer(missing_values=np.NaN,strategy='median')
imputer=median_imputer.fit(train_X)
v_train_X=imputer.transform(train_X)
imputerval=median_imputer.fit(val_X)
v_val_X=imputerval.transform(val_X)
train_X=pd.DataFrame(v_train_X, columns=train_X.columns,index=train_X.index)
val_X=pd.DataFrame(v_val_X, columns=val_X.columns,index=val_X.index)

In [ ]:
cbdummy=cbmodel.fit(train_X, train_y)

In [ ]:
dictdummy=cbdummy.get_all_params()

In [ ]:
lr=dictdummy['learning_rate']*2

we doubled the CatBoost calculated learning rate for speed

Preliminary CatBoost classifier: Bayesian hyperparameter optimization with Optuna.
All is done in the training set.

In [ ]:
cb_auc=[]
cb_param=[]

def objective(trial):
    train_X, val_X, train_y, val_y = train_test_split(X_train, y_train, test_size=0.2)
    #again 0.8:0.2 inner split of the training set
    
    median_imputer=SimpleImputer(missing_values=np.NaN,strategy='median')
    imputer=median_imputer.fit(train_X)
    v_train_X=imputer.transform(train_X)
    imputerval=median_imputer.fit(val_X)
    v_val_X=imputerval.transform(val_X)
    train_X=pd.DataFrame(v_train_X, columns=train_X.columns,index=train_X.index)
    val_X=pd.DataFrame(v_val_X, columns=val_X.columns,index=val_X.index)
    #again median imputation procedure
    
    param={
        'verbose':1000,
        'eval_metric':'AUC',
        'depth' : trial.suggest_int('depth', 4, 10),
        'objective' : trial.suggest_categorical('objective', ['Logloss', 'CrossEntropy']),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.01, 0.1),
        'l2_leaf_reg':trial.suggest_loguniform('l2_leaf_reg', 3, 30),
        'boosting_type':'Ordered',
        'bootstrap_type':'Bayesian',
        'bagging_temperature' : trial.suggest_uniform('bagging_temperature', 0, 4),
        'eta':lr,
        'iterations':700,
    }
    if param['objective'] == 'Logloss':
        param['random_strength'] = trial.suggest_uniform('random_strength', 0.5, 5)
    #hyperparameter optimization list.
    #The obiective is varied only to obtain a binary choice between fixed and optimizable random_strength.
    
    cbmodel = CatBoostClassifier(**param)
                                                                                                 
    cbmodel.fit(train_X, train_y)
    predictioncb = cbmodel.predict_proba(val_X)
    auccb = roc_auc_score(val_y, cbmodel.predict_proba(val_X) [:,1])
    
    roc_auc_score(val_y, cbmodel.predict_proba(val_X) [:,1])
        
    return auccb
    

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=300)
    print(study.best_trial)
 

now pick the best model

In [ ]:
params_cb_preliminary=study.best_trial.params

In [ ]:
params_cb_preliminary['verbose'] = 1000

In [ ]:
cb_preliminary = CatBoostClassifier(**params_cb_preliminary)

Start BorutaSHAP feature selection procedure, with the optimized CatBoost preliminary model


the BorutaSHAP procedure can be exceedingly long. Some suggestion to shorten it:

reduce number of trials. 800 is a very high number. You can use TentativeRoughFix if uncertain features trouble you.

you could change the voting from 6 over 8 to 5 over 5 or 4 over 5 or whatever. It depends from your dataset and the Shapley value distribution of it.

you could optimize the preliminary mode on a different hyperparameter space. For example, you could restrict the number of leaves to 4-6.

In any case, you should check for consistency of conserved/eliminated features between different training/validation folds.

In [ ]:
optimiz_cb= cb_preliminary

def cb_boruta(optimizer):
    featurelist=[]

    cv = StratifiedKFold(n_splits=8, shuffle=True)
    #Eight stratified split of the training set

    for train_index, test_index in cv.split(X_train,y_train):

        trainX = X_train.iloc[lambda x: train_index]
        testX = X_train.iloc[lambda x: test_index]
        trainy = np.take(y_train, train_index)
        testy = np.take(y_train, test_index)
        
        median_imputer = SimpleImputer(missing_values = np.NaN,strategy = 'median')
        imputer = median_imputer.fit(trainX)
        vtrainX = imputer.transform(trainX)
        imputertest = median_imputer.fit(testX)
        vtestX = imputertest.transform(testX)
        trainX = pd.DataFrame(vtrainX, columns = trainX.columns,index = trainX.index)
        testX = pd.DataFrame(vtestX, columns = testX.columns,index = testX.index)
        
        #for each fold apply the post-split median imputation procedure
        
        Feature_Selector = BorutaShap(model = optimizer,
                                  importance_measure = 'shap',
                                  percentile = 85, 
                                  pvalue = 0.1,
                                  classification = True)
        
        #weak BorutaShap feature selector. 
        #Percentile is lowered, pvalue is doubled, in respect to default.
        
        Feature_Selector.fit(trainX, trainy, n_trials=800, random_state=0)
        #Feature_Selector.TentativeRoughFix()
        #Uncomment the line above if you want no uncertain feature.
        Feature_Selector.plot(X_size=12, figsize=(16,12),
                y_scale='log', which_features='all')
        #For the paper we plotted only the selected features.
        
        strainX = Feature_Selector.Subset()
        selected = [x for x in strainX.columns]
        print('features selected',selected)
        rejected=Feature_Selector.rejected
        featurelist.extend(rejected)
        stestX = testX[selected]
        optimizer.fit(strainX,trainy)
        
        rocboruta=roc_auc_score(testy, optimizer.predict_proba(stestX)[:,1])
        
        print(rocrocboruta)
        
    return featurelist
      
featsel=cb_boruta(optimiz_cb)

Now use a voting procedure to eliminate the features eliminated in six folds over eight.

Eliminated features for each fold are appended in the list featsel.

In [ ]:
feadrop=[feature for feature in featsel if featsel.count(feature)>=6]

In [ ]:
feadrop=list(set(feadrop))

In [ ]:
feadrop

drop eliminated features from the training/validation/test sets.

In [ ]:
X_trainshort=X_train.drop(feadrop, axis=1)
X_valshort=X_valid.drop(feadrop, axis=1)
X_testshort=X_test.drop(feadrop, axis=1)

END OF FEATURE SELECTION

CATBOOST MODEL ON REDUCED FEATURE SPACE

Is there any feature that could be worthy of a finer quantization in CatBoost (Golden feature)?
For example, in our model we tried both the first CT extracted feature and the PO2/FiO2

In [ ]:
gf1=X_trainshort.columns.get_loc("PaO2/FiO2")
gf2=X_trainshort.columns.get_loc("Feature_CT_1")

create the categorical options for quantization of golden features.

In [ ]:
s1=str(gf1)+':border 1024'

In [ ]:
s2=str(gf2)+':border 1024'

In [ ]:
s3=str(gf1)+':border 254'

In [ ]:
s4=[s1,s2]

...back to the original CatBoost selected learning rate 

In [ ]:
lr=lr/2

Optuna CatBoost hyperparameter optimization on reduced feature space. Again, in the training set.
The goal is to create a shortlist of hyperparameter configurations to be checked on the validation set with the overfitting detector

In [ ]:
#same as optuna hyperparameter optimization above

cb_auc=[]
cb_param=[]

def objective(trial):
    train_X, val_X, train_y, val_y = train_test_split(X_trainshort, y_train, test_size=0.2)
    #again 0.8:0.2 inner split of the training set
    median_imputer=SimpleImputer(missing_values=np.NaN,strategy='median')
    imputer=median_imputer.fit(train_X)
    v_train_X=imputer.transform(train_X)
    imputerval=median_imputer.fit(val_X)
    v_val_X=imputerval.transform(val_X)
    train_X=pd.DataFrame(v_train_X, columns=train_X.columns,index=train_X.index)
    val_X=pd.DataFrame(v_val_X, columns=val_X.columns,index=val_X.index)
    #usual median imputation procedure 
    
    
    param={
        'verbose':1000,
        'eval_metric':'AUC',
        'depth' : trial.suggest_int('depth', 4, 10),
        'objective' : trial.suggest_categorical('objective', ['Logloss', 'CrossEntropy']),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.01, 0.1),
        'l2_leaf_reg':trial.suggest_loguniform('l2_leaf_reg', 3, 50),
        'boosting_type':'Ordered',
        'bootstrap_type': 'Bayesian',
        'per_float_feature_quantization':trial.suggest_categorical('per_float_feature_quantization', ['13:border_count=1024','18:border_count=1024','18:border_count=254',['13:border_count=1024', '18:border_count=1024']]),                                           
        'eta':lr,
        'iterations':700,
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0, 4)
    }
    #it seems that you need to specify categorical suggestions actually writing them,
    #e.g. copying s1, s2, s3, s4 as '13:border_count=1024','18:border_count=1024','18:border_count=254',['13:border_count=1024', '18:border_count=1024']
    
    if param['objective'] == 'Logloss':
        param['random_strength'] = trial.suggest_uniform('random_strength', 0.5, 5)

    cbmodel = CatBoostClassifier(**param)
                                                 
                                                    
    cbmodel.fit(train_X, train_y)
    predictioncb = cbmodel.predict_proba(val_X)
    auccb = roc_auc_score(val_y, cbmodel.predict_proba(val_X) [:,1])
    
    
    return auccb
    

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=300)
    print(study.best_trial)
 


In [ ]:
optuna.visualization.plot_optimization_history(study)

put models in a dataframe

In [ ]:
dfs=study.trials_dataframe()

shortlist models picking only AUC larger than a threshold value

alternative: pick the best ten models..

In [ ]:
shortlist=dfs.loc[dfs['value'] >= 0.96]

In [ ]:
shortlength=len(shortlist)

define the hyperparameters needed

In [ ]:
numbers=[x for x in shortlist['number']]

In [ ]:
dictparam={}

In [ ]:
for x in numbers:
    dictparam[x]={'verbose':1000,'depth' : shortlist.loc[x,'params_depth'],
        'objective' : shortlist.loc[x,'params_objective'],
        'colsample_bylevel': shortlist.loc[x,'params_colsample_bylevel'],
        'l2_leaf_reg':shortlist.loc[x,'params_l2_leaf_reg'],
        'boosting_type':'Ordered',
        'bootstrap_type': 'Bayesian',
        'per_float_feature_quantization':shortlist.loc[x,'params_per_float_feature_quantization'],                                           
        'eta':0.008,
        'iterations':20000,
        'bagging_temperature': shortlist.loc[x,'params_bagging_temperature'],
        'random_strength': shortlist.loc[x,'params_random_strength']}

learning rate fixed to a number slightly lower than the lr precedently used

number of iterations fixed to a large number so to use overfitting detector

clean from nan (possible for random_strength)

In [ ]:
dict_param = {k: {a: b for a, b in v.items() if not b!=b} for k, v in dictparam.items()}

In [ ]:
cb_optimized = [CatBoostClassifier(**clean_param[x]) for x in numbers]

usual median imputing

In [ ]:
median_imputer=SimpleImputer(missing_values=np.NaN,strategy='median')
imputer=median_imputer.fit(X_trainshort)
vXtrainshort=imputer.transform(X_trainshort)
Xmtrain=pd.DataFrame(vXtrainshort, columns=X_trainshort.columns,index=X_trainshort.index)

In [ ]:
median_imputer = SimpleImputer(missing_values=np.NaN,strategy='median')
imputer = median_imputer.fit(X_valshort)
vXvalshort = imputer.transform(X_valshort)
Xmvalid=pd.DataFrame(vXvalshort, columns=X_valshort.columns,index=X_valshort.index)

use Pool to select best iteration number with overfitting detector

In [ ]:
eval_dataset = Pool(data=Xmvalid,
                    label=y_valid)

fit shortlisted models to validation set, using overfitting detector

In [ ]:
itercb=[]

In [ ]:
auccb=[]

In [ ]:
for i in range(shortlength):
    cb_optimized[i].fit(Xmtrain, y_train,eval_set=eval_dataset,use_best_model=True)
    itera=cb_optimized[i].get_best_iteration()
    score = roc_auc_score(y_valid, cb_optimized[i].predict_proba(Xmvalid) [:,1])
    itercb.append(itera)
    auccb.append(score)

In [ ]:
best_valid_auc=max(auccb)

In [ ]:
best_valid_index = auccb.index(best_valid_auc)

In [ ]:
best_valid_itera = itercb[best_valid_index]

In [ ]:
best_val_param=cb_optimized[best_valid_index].get_params()

In [ ]:
best_val_param['iterations']=best_valid_itera

In [ ]:
best_val_param

auc, best iteration number and parameters of the best model on validation set

END OF CATBOOST MODEL ON REDUCED FEATURE SPACE

In [ ]:
RETRAINING AND TEST

if the best validation model is indeed the best of all folds, retrain on joined training +validation

and then evaluate on the test

create the joined training+validation

In [ ]:
Xshorttrainval=pd.concat([X_trainshort, X_valshort], axis=0, sort=False)

In [ ]:
yvaltrain=np.concatenate((y_train,y_valid),axis=0)

do again the median imputation

In [ ]:
median_imputer = SimpleImputer(missing_values=np.NaN,strategy='median')
imputer = median_imputer.fit(Xshorttrainval)
vXtrainvalshort = imputer.transform(Xshorttrainval)
Xmtrainval=pd.DataFrame(vXtrainvalshort, columns=Xshorttrainval.columns,index=Xshorttrainval.index)

In [ ]:
median_imputer = SimpleImputer(missing_values=np.NaN,strategy='median')
imputer = median_imputer.fit(X_testshort)
vXtestshort = imputer.transform(X_testshort)
Xmtest=pd.DataFrame(vXtestshort, columns=X_testshort.columns,index=X_testshort.index)

since the train+validation set is larger, use 120% of the number of iterations

In [ ]:
final_param=best_val_param

In [ ]:
final_param['iterations']=int(final_param['iterations']*1.2)

In [ ]:
cb_final=CatBoostClassifier(**final_param)

train it on the joined set

In [ ]:
cbfinal.fit(Xmtrainval, yvaltrain)

evaluate it on the test set

In [ ]:
aucfinal = roc_auc_score(y_test, cb_final.predict_proba(Xmtest) [:,1])

do some plot

In [ ]:
model = cb_final

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(Xmtest)

SHAP force plot for patient 6

In [ ]:
i = 6
shap.initjs()

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[i,:], Xmtest.iloc[i,:], link="logit", show=False)